In [1]:
import pandas as pd
import numpy as np  


In [2]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [11]:
df = pd.read_csv(r'C:\Users\samco\OneDrive\LP\Repositorios DEV\EBAC_Estudos\EBAC\Tratamento_dados\clientes_remove_outliers.csv')

print(df)

                         nome             cpf  idade        data  \
0     Sr. Vitor Gabriel Silva  640.719.385-01     90  1933-12-06   
1               Calebe Santos  651.297.384-46     40  1984-03-06   
2          Maria Julia Castro  164.750.293-43     76  1947-04-12   
3              Arthur Da Rosa  085.167.249-30     53  1970-04-16   
4                Juan Pereira  520.876.319-21     68  1955-10-25   
...                       ...             ...    ...         ...   
1005            Eloah Peixoto  064.917.253-16     79  1944-05-23   
1006              Luiza Sales  726.431.850-80     55  1968-05-01   
1007      Dr. Alexandre Sales  427.398.061-40     87  1936-06-05   
1008            Alícia Campos  162.790.485-94     49  1975-03-20   
1009              Luigi Porto  826.509.374-83     43  1980-08-18   

                                                                            endereco  \
0                        trevo emanuel silva, 482\nvila tirol\n47847085 pereira / ap   
1      

Mascarar dados pessoais

In [4]:
#Altera a exibição de todos os cpfs mostrando só os primeiros 3 números '{cpf[:3]}', jogos de asteriscos '-***.***-' e os ultimos 2 {cpf[-2:]}
df['cpf_mascara'] = df['cpf'].apply(lambda cpf: f'{cpf[:3]}.***.***-{cpf[-2:]}')

Corrigir datas  

In [5]:
df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d', errors='coerce')

data_atual = pd.to_datetime('today')

#Se a data cadastrada for <= a data atual vai atualizar para '1900-01-01'
df['data_atualizada'] = df['data'].where(df['data'] <= data_atual, pd.to_datetime('1900-01-01'))

#Ajustar a idade de acordo com a data atual - data atualizada de aniversario da pessoa
df['idade_ajustada'] = data_atual.year - df['data_atualizada'].dt.year

#Ajustar se o mes atual for menor do que o mês do aniversario e o dia atual for menor que o dia do aniversario (já fez aniversario) vai retornar 1 e vai somar á idade ajustada
df['idade_ajustada'] -= ((data_atual.month <= df['data_atualizada'].dt.month) & (data_atual.day < df['data_atualizada'].dt.day)).astype(int)

#Atribuir valor nulo às idades ajustadas que ficarem maiores que 100, pois estariam erradas
df.loc[df['idade_ajustada'] > 100, 'idade_ajustada'] = np.nan #atribui o valor nulo


Corrigir campo com múltiplas informações

In [6]:
#Separa o endereço pelos \n e pega cada parte para dividir em campos diferentes e utiliza o if para cadastrar como desconhecido os endereços que tem menos de 2 campos(\n)
df['endereco_curto'] = df['endereco'].apply(lambda x: x.split('\n')[0].strip())
df['bairro'] = df['endereco'].apply(lambda x: x.split('\n')[1].strip() if len(x.split('\n')) > 1 else 'Desconhecido')
df['estado_sigla'] = df['endereco'].apply(lambda x: x.split(' / ')[-1].strip() if len(x.split('\n')) > 1 else 'Desconhecido')


Verificando a formatação de endereços

In [7]:
#Vai alterar o campo endereço para 'Endereço inválido' se o endereco for maior que 50 caracteres ou menor que 5
df['endereco_curto'] = df['endereco_curto'].apply(lambda x: 'Endereço inválido' if len(x) >50 or len(x) < 5 else x)

Corrigir dados errôneos

In [8]:
#Altera o campo para 'CPF inválido' todos os cpfs que o tamanho for diferente de 14 caracteres
df['cpf'] = df['cpf'].apply(lambda x: x if len(x) == 14 else 'CPF inválido')

#Lista de estados BR
estados_br = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']

#Altera o campo para desconhecido se o valor for diferente de algum dos que tem na lista
df['estado_sigla'] = df['estado_sigla'].str.upper().apply(lambda x: x if x in estados_br else 'Desconhecido')


Renomeando os nomes dos campos para o que era

In [9]:
df['cpf'] = df['cpf_mascara']
df['idade'] = df['idade_ajustada']
df['endereco'] = df['endereco_curto']
df['estado'] = df['estado_sigla']
df_salvar = df[['nome', 'cpf', 'idade', 'data', 'endereco', 'bairro', 'estado']]
df_salvar.to_csv('clientes_tratados.csv', index=False)

In [10]:
print('Novo Dataframe tratado: \n', pd.read_csv('clientes_tratados.csv'))

Novo Dataframe tratado: 
                          nome             cpf  idade        data  \
0     Sr. Vitor Gabriel Silva  640.***.***-01   93.0  1933-12-06   
1               Calebe Santos  651.***.***-46   42.0  1984-03-06   
2          Maria Julia Castro  164.***.***-43   79.0  1947-04-12   
3              Arthur Da Rosa  085.***.***-30   55.0  1970-04-16   
4                Juan Pereira  520.***.***-21   70.0  1955-10-25   
...                       ...             ...    ...         ...   
1005            Eloah Peixoto  064.***.***-16   81.0  1944-05-23   
1006              Luiza Sales  726.***.***-80   58.0  1968-05-01   
1007      Dr. Alexandre Sales  427.***.***-40   90.0  1936-06-05   
1008            Alícia Campos  162.***.***-94   50.0  1975-03-20   
1009              Luigi Porto  826.***.***-83   45.0  1980-08-18   

                        endereco                     bairro estado  
0       trevo emanuel silva, 482                 vila tirol     AP  
1              reca